In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
import numpy as np
import sklearn
import statistics
import math 
import random

In [31]:
data=pd.read_csv("/content/drive/My Drive/sc_lab/Iris2.csv")
data=pd.DataFrame(data)
data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [32]:
d1=data[["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
d2=data["Species"]
d1=np.array(d1)
d2=np.array(d2)

In [33]:
class_to_no={}
class_to_no['Iris-setosa']=0
class_to_no['Iris-versicolor']=1
class_to_no['Iris-virginica']=2

In [34]:
no_to_class={}
no_to_class[0]='Iris-setosa'
no_to_class[1]='Iris-versicolor'
no_to_class[2]='Iris-virginica'

In [35]:

def get_group(x,y):
  group=[[[] for i in range(4)] for i in range(3)]
  for i in range(len(x)):
    for j in range(4):
      group[class_to_no[y[i]]][j].append(x[i][j])
  return group



In [36]:

def get_group_mean(group):
  group_mean=[[] for i in range(3)]
  for i in range(len(group)):
    for j in range(len(group[i])):
      if(len(group[i][j])==0):
        group_mean[i].append(0)
      else:
        group_mean[i].append(statistics.mean(group[i][j]))

  return group_mean



In [37]:

def get_stddev(group):
  group_stdev=[[] for i in range(3)]
  for i in range(len(group)):
    for j in range(len(group[i])):
      if(len(group[i][j])<2):
        group_stdev[i].append(0)
      else:
        group_stdev[i].append(statistics.stdev(group[i][j]))
  
  return group_stdev



In [38]:
def prob(x,mean,stddev):
  if(stddev==0):
    return 0
  return (math.exp(-((x-mean)**2)/(2*(stddev**2))))/(math.sqrt(2*math.pi)*stddev)

In [39]:

def get_count_class(y):
  count_class={}
  count_class['Iris-setosa']=0
  count_class['Iris-versicolor']=0
  count_class['Iris-virginica']=0
  for i in y:
    count_class[i]+=1
  return count_class

In [40]:

def get_prob_class(count_class,y):
  prob_class={}
  for i in count_class:
    prob_class[i]=count_class[i]/len(y)
  return prob_class

In [41]:
def predict(group,group_mean,group_stdev,prob_class,x_test):
  predict_class=[]
  for i in range(len(x_test)):

    temp=[]
    for j in range(len(group)):
      p=1
      for k in range(4):
        p=p*prob(x_test[i][k],group_mean[j][k],group_stdev[j][k])
      p=p*prob_class[no_to_class[j]]
      temp.append(p)
    
    y=0
    x=0
  
    for i in range(len(temp)):
      if(temp[i]>x):
        y=i
        x=temp[i]

    predict_class.append(y)
  return predict_class  
  


In [42]:
def classification_report(cm):
    accuracy = 0
    total = np.sum(cm)
    diagonal = np.sum(np.diag(cm))
    accuracy=(diagonal/total)*100
    print("Accuracy : => ",(diagonal/total)*100)
    return accuracy

In [44]:
Avg_accuracy=0
block=int(len(data)/10)
M=[[ 0 for i in range(3)] for i in range(3)]
for i in range(10):
  temp_x_data=[]
  temp_y_data=[]
  
  x_test=[]
  y_test=[]
  

  point=[]
  for j in range(block):
    point.append(block*i+j)

  
  for j in range(len(data)):
    if(j in point):
      x_test.append(d1[j])
      y_test.append(d2[j])
    else:
      temp_x_data.append(d1[j])
      temp_y_data.append(d2[j])
  
  group=get_group(temp_x_data,temp_y_data)
  
  group_mean=get_group_mean(group)
  group_stdev=get_stddev(group)
  count_class=get_count_class(temp_y_data)
  prob_class=get_prob_class(count_class,temp_y_data)
  
  predict_class=predict(group,group_mean,group_stdev,prob_class,x_test)
  print("Fold ",i+1)
  # for j in range(len(predict_class)):
  #   print(no_to_class[predict_class[j]],y_test[j])
  
  matrix=[[0 for j in range(3)] for j in range(3)]

  for j in range(len(x_test)):
    matrix[class_to_no[y_test[j]]][predict_class[j]]+=1

  for p in range(3):
    for q in range(3):
      M[p][q]+=matrix[p][q]

  
  print("Confusion matrix")
  print(matrix)

  accu=classification_report(matrix)
  
  Avg_accuracy+=accu
  print()

Avg_accuracy/=10


print("Final confusion matrix")
print(M)
print()
print("Average accuracy=>",Avg_accuracy)

Fold  1
Confusion matrix
[[15, 0, 0], [0, 0, 0], [0, 0, 0]]
Accuracy : =>  100.0

Fold  2
Confusion matrix
[[15, 0, 0], [0, 0, 0], [0, 0, 0]]
Accuracy : =>  100.0

Fold  3
Confusion matrix
[[15, 0, 0], [0, 0, 0], [0, 0, 0]]
Accuracy : =>  100.0

Fold  4
Confusion matrix
[[5, 0, 0], [0, 9, 1], [0, 0, 0]]
Accuracy : =>  93.33333333333333

Fold  5
Confusion matrix
[[0, 0, 0], [0, 14, 1], [0, 0, 0]]
Accuracy : =>  93.33333333333333

Fold  6
Confusion matrix
[[0, 0, 0], [0, 13, 2], [0, 0, 0]]
Accuracy : =>  86.66666666666667

Fold  7
Confusion matrix
[[0, 0, 0], [0, 10, 0], [0, 0, 5]]
Accuracy : =>  100.0

Fold  8
Confusion matrix
[[0, 0, 0], [0, 0, 0], [0, 2, 13]]
Accuracy : =>  86.66666666666667

Fold  9
Confusion matrix
[[0, 0, 0], [0, 0, 0], [0, 2, 13]]
Accuracy : =>  86.66666666666667

Fold  10
Confusion matrix
[[0, 0, 0], [0, 0, 0], [0, 0, 15]]
Accuracy : =>  100.0

Final confusion matrix
[[50, 0, 0], [0, 46, 4], [0, 4, 46]]

Average accuracy=> 94.66666666666666
